In [ ]:
import requests
import polars as pl
import json
from box import Box
from IPython.display import display

In [ ]:
def get_size_of_encode():
    url = 'https://www.encodeproject.org/search/?type=Experiment&format=json&limit=all'
    headers = {'accept': 'application/json'}
    
    response = requests.get(url, headers=headers)
    data = response.json()
    experiment_graph = data.get('@graph', [])
    number_of_experiments = len(experiment_graph)
    number_of_files = 0
    
    for experiment in experiment_graph:
        experiment_files = experiment.get('files', [])
        experiment_file_count = len(experiment_files)
        number_of_files += experiment_file_count
    
    return number_of_experiments, number_of_files

In [ ]:
number_of_experiments, number_of_files = get_size_of_encode()
print(f"Total number of experiments on ENCODE: {number_of_experiments}")
print(f"Total number of files ENCODE: {number_of_files}")

In [ ]:
## Le broken ...

def all_files_to_static_json():
    url = 'https://www.encodeproject.org/search/?type=File&format=json&limit=all'
    headers = {'accept': 'application/json'}
    response = requests.get(url, headers=headers)
    data = response.json()
    file_graph = data.get('@graph', [])
    with open('encode_files.json', 'w') as f:
        json.dump(file_graph, f)
        
    print('File graph saved to encode_files.json')

In [ ]:
import pickle

with open('../accessions.pkl', 'rb') as f:
    accessions = pickle.load(f)
    
print(len(accessions))

In [ ]:
encode = pl.read_parquet('../encode_files.parquet')
display(encode)
# print(encode.select("biological_replicates"))
# display(encode)
for column in encode.iter_columns():
    display(encode.select(column).unique())
    
print(encode.select("output_type").unique().to_list())


title,accession,dataset,assembly,technical_replicates,biological_replicates,file_format,file_type,file_format_type,file_size,assay_term_name,term_name,organ_slims,simple_biosample_summary,origin_batches,label,download_url,derived_from,genome_annotation,paired_end,paired_with,preferred_default,run_type,read_length,mapped_read_length,cropped_read_length,cropped_read_length_tolerance,mapped_run_type,read_length_units,output_category,output_type,index_of,quality_metrics,lab_title,project,step_run,date_created,analysis_step_version,restricted,submitter_comment,status,annotation_type,annotation_subtype,biochemical_inputs,encyclopedia_version
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""ENCFF566NGG""","""ENCFF566NGG""","""/experiments/ENCSR369MDF/""","""GRCh38""","""['1_1']""","""[1]""","""bam""","""bam""","""NA""","""3936091129""","""RNA-seq""","""K562""","""['blood', 'bodily fluid']""","""treated with 5 μM JQ1 for 4 ho…","""['/biosamples/ENCBS291NHT/']""","""NA""","""https://encodeproject.org/file…","""['/files/ENCFF598IDH/', '/file…","""V29""","""NA""","""NA""","""NA""","""NA""","""NA""","""150""","""NA""","""NA""","""paired-ended""","""nt""","""alignment""","""transcriptome alignments""","""NA""","""[{'assay_term_name': 'RNA-seq'…","""ENCODE Processing Pipeline""","""ENCODE""","""{'status': 'released', 'lab': …","""2021-12-31T08:51:07.433754+00:…","""{'software_versions': [{'submi…","""NA""","""NA""","""released""","""NA""","""NA""","""NA""","""NA"""
"""ENCFF642UFR""","""ENCFF642UFR""","""/experiments/ENCSR250SCW/""","""mm10""","""['2_1']""","""[2]""","""tar""","""tar""","""NA""","""4117922""","""single-cell RNA sequencing ass…","""left cerebral cortex""","""['brain']""","""male adult (2 months) strain B…","""NA""","""NA""","""https://encodeproject.org/file…","""['/files/ENCFF723SJO/', '/file…","""M21""","""NA""","""NA""","""True""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""quantification""","""unfiltered sparse gene count m…","""NA""","""[{'assay_term_name': 'single-c…","""Barbara Wold, Caltech""","""ENCODE""","""{'status': 'released', 'date_c…","""2022-01-30T04:31:33.614985+00:…","""{'date_created': '2021-12-14T2…","""NA""","""NA""","""released""","""NA""","""NA""","""NA""","""NA"""
"""ENCFF094MID""","""ENCFF094MID""","""/experiments/ENCSR414JJE/""","""NA""","""['1_2']""","""[1]""","""fastq""","""fastq""","""NA""","""263567745""","""Mint-ChIP-seq""","""naive B cell""","""['blood', 'bodily fluid']""","""female adult (39 years)""","""['/biosamples/ENCBS517ZRS/']""","""H3K27me3""","""https://encodeproject.org/file…","""NA""","""NA""","""2""","""/files/ENCFF481AOK/""","""NA""","""paired-ended""","""58""","""NA""","""NA""","""NA""","""NA""","""nt""","""raw data""","""reads""","""NA""","""[]""","""Bradley Bernstein, Broad""","""ENCODE""","""NA""","""2022-02-11T14:58:17.561304+00:…","""NA""","""NA""","""NA""","""released""","""NA""","""NA""","""NA""","""NA"""
"""ENCFF506TJN""","""ENCFF506TJN""","""/experiments/ENCSR369MDF/""","""GRCh38""","""['2_1']""","""[2]""","""bigWig""","""bigWig""","""NA""","""120998658""","""RNA-seq""","""K562""","""['blood', 'bodily fluid']""","""treated with 5 μM JQ1 for 4 ho…","""['/biosamples/ENCBS798JQV/']""","""NA""","""https://encodeproject.org/file…","""['/files/ENCFF124OPS/', '/file…","""V29""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""NA""","""signal""","""signal of all reads""","""NA""","""[]""","""ENCODE Processing Pipeline""","""ENCODE""","""{'status': 'released', 'lab': …","""2021-12-31T08:51:14.572055+00:…","""{'analysis_step': {'documents'…","""NA""","""NA""","""released""","""NA""","""NA""","""NA""","""NA"""
"""ENCFF174IGS""","""ENCFF174IGS""","""/experiments/ENCSR040TXN/""","""NA""","""['1_2']""","""[1]""","""fastq""","""fastq""","""NA""","""45801960""","""Mint-ChIP-seq""","""central memory CD

title
str
"""ENCFF000CLE"""
"""ENCFF474MDU"""
"""ENCFF791PXN"""
"""ENCFF253SCW"""
"""ENCFF777RLP"""
…
"""ENCFF907HKK"""
"""ENCFF251SUM"""
"""ENCFF261PYX"""


accession
str
"""ENCFF508JIW"""
"""ENCFF015YQN"""
"""ENCFF674VOZ"""
"""ENCFF730DGT"""
"""ENCFF872EUN"""
…
"""ENCFF144KQA"""
"""ENCFF743IZN"""
"""ENCFF000XFW"""


dataset
str
"""/experiments/ENCSR116YMU/"""
"""/experiments/ENCSR597XRK/"""
"""/experiments/ENCSR813BDU/"""
"""/experiments/ENCSR800MYV/"""
"""/experiments/ENCSR187IVR/"""
…
"""/experiments/ENCSR325JLI/"""
"""/experiments/ENCSR096USV/"""
"""/experiments/ENCSR000DTA/"""


assembly
str
"""NA"""
"""GRCh38-minimal"""
"""ce11"""
"""mm10-minimal"""
"""mm10"""
…
"""ce10"""
"""dm6"""
"""mm9"""


...

status
str
"""released"""
"""archived"""


annotation_type
str
"""NA"""


annotation_subtype
str
"""NA"""


biochemical_inputs
str
"""NA"""


encyclopedia_version
str
"""NA"""


In [28]:
print(encode.select("output_type").unique().to_series().to_list())

['normalized signal of all reads', 'gene alignments', 'microRNA quantifications', 'intensity values', 'control normalized signal', 'base overlap signal', 'redacted unfiltered alignments', 'transcript quantifications', 'rejected reads', 'DHS peaks', 'percentage normalized signal', 'subreads', 'transcription segment quantifications', 'plus strand signal', 'raw signal', 'signal of unique reads', 'filtered transcribed fragments', 'filtered reads', 'raw imaging signal', 'exon quantifications', 'clusters', 'chromatin stripes', 'peaks and background as input for IDR', 'reporter code counts', 'contact domains', 'transcriptome alignments', 'unfiltered modified peptide quantification', 'contact matrix', 'polyA sites', 'nested contact domains', 'minus strand signal of all reads', 'idat red channel', 'plus strand signal of all reads', 'pairs', 'genome subcompartments', 'replicated peaks', 'methylation state at CHG', 'predicted transcription start sites', 'IDR thresholded peaks', 'IDR ranked peaks'